In [1]:
import numpy as np
import pandas as pd
import torch

In [2]:
import os

In [3]:
import pickle

In [4]:
import pylangacq as pla

In [5]:
import random

Load Targets

In [6]:
# y_cookie = torch.load('cookie_data_targets.pt')

In [6]:
y_cookie = torch.load('cookie_target_dict.pt')

In [8]:
len(y_cookie)

552

In [9]:
# y_sentence = torch.load('sentence_data_targets.pt')

In [7]:
y_sentence = torch.load('sentence_target_dict.pt')

In [11]:
len(y_sentence)

240

Load Samples

In [8]:
X_cookie_filenames = torch.load('cookie_fnames_dict.pt')

In [13]:
len(X_cookie_filenames)

552

In [9]:
X_sent_filenames = torch.load('sentence_fnames_dict.pt')

In [15]:
len(X_sent_filenames)

240

In [10]:
num_samples = len(X_cookie_filenames) + len(X_sent_filenames)

In [17]:
num_samples

792

In [11]:
X_cookie_reader = pla.read_chat('./Pitt/*/cookie/*.cha')
X_sentence_reader = pla.read_chat('./Pitt/*/sentence/*.cha')

-----------------------------------

Missing data -- targets

group missing or input error st no mean to replace with and left untouched in target generation

replace with closest analog as per means generated from target generation:

cookie {'Control': 30, 'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 20, 'Probable': 19, 'Other': 24}

sentence {'ProbableAD': 19, 'MCI': 28, 'Memory': 30, 'Vascular': 17, 'PossibleAD': 21, 'Control': 30, 'Probable': 19, 'Other': 24}

In [19]:
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])

In [21]:
target_674 = X_sentence_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/sentence/236-0.cha']['Participants']['PAR']['group']

In [143]:
target_674

''

In [144]:
group_674

'Dementia'

In [154]:
# Replace Dementia with Possible mean (as per diagnostic codes in Pitt-data.xlxs and accompanying Pitt-readme.pdf)
y_sentence[674] = 21

In [139]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha
219 
/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha
511 possibleAD


In [146]:
target_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['education']
group_219 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Control/cookie/304-1.cha']['Participants']['PAR']['group']

In [147]:
target_219

''

In [148]:
group_219

''

In [156]:
# Replace '' with Control mean (as per folder and dx codes in .xlxs)
y_cookie[219] = 30

In [149]:
target_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['education']
group_511 = X_cookie_reader.headers()['/Users/renee/Documents/WIS_Spr20/DL/FinalProj/Pitt/Dementia/cookie/585-0.cha']['Participants']['PAR']['group']

In [150]:
target_511

''

In [151]:
group_511

'possibleAD'

In [158]:
# Replace possibleAD with PossibleAD mean
y_cookie[511] = 20

In [159]:
# confirm all targets filled now 
for sent_k in y_sentence:
    if type(y_sentence[sent_k]) == str:
        print(X_sent_filenames[sent_k])
        print(sent_k,y_sentence[sent_k])


In [160]:
for coo_kie in y_cookie:
    if type(y_cookie[coo_kie]) == str:
        print(X_cookie_filenames[coo_kie])
        print(coo_kie,y_cookie[coo_kie])

In [161]:
# torch.save(y_cookie,'cookie_target_dict.pt')

In [162]:
# torch.save(y_sentence,'sentence_target_dict.pt')

---------------------------------------------------------------

Load Vocab Embeddings

In [12]:
glove_path = './PretrainedWordEmb/'

In [13]:
vocab_emb = pickle.load(open(f'{glove_path}/addb.vocab_emb.glove.42B.300.pkl', 'rb'))

In [33]:
len(vocab_emb)

2188

In [14]:
pos_dict = torch.load('pos_dict.pt')

In [267]:
num_samples

792

In [59]:
# valid_ix = random.sample(range(num_samples),int(.20*num_samples))

In [61]:
# train_ix = [i for i in range(num_samples) if i not in valid_ix]

In [63]:
len(valid_ix)

158

In [62]:
len(train_ix)

634

In [65]:
# torch.save(valid_ix,'valid_ix.pt')

In [67]:
# torch.save(train_ix,'train_ix.pt')

In [72]:
valid_ix = torch.load('valid_ix.pt')

In [73]:
len(valid_ix)

158

In [74]:
train_ix = torch.load('train_ix.pt')

In [75]:
len(train_ix)

634

In [62]:
def get_item(ix=None,valid=False):
    
    # first should be...all zeros: each index represents greater than .. but you cant have a zero score..
    def num_vectorize(t):

        v = torch.zeros([29])
        v[:t-1] = 1

        return v.double()
    
    ix = random.randint(1,num_samples-1) if ix is None else ix
    
    #index, filename
    if ix in X_cookie_filenames.keys():
        _file = (ix,X_cookie_filenames[ix])
        _reader = X_cookie_reader
        _targetdict = y_cookie
        
    else:
        _file = (ix,X_sent_filenames[ix])
        _reader = X_sentence_reader
        _targetdict = y_sentence
        
    file,data,targetdict = (_file,_reader,_targetdict)
    
    embeddings = []
    targets = []

    embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file[1]] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file[1]] for (tokensraw,pos,tokenstem,dependency) in utterance])]
    target = targetdict[ix]

    for tkn in embedding:

        tkn[1][tkn[2]] = 1
#     print(target)   
    if valid: return [torch.cat((tkn[0],tkn[1])) for tkn in embedding], torch.tensor(target)
    return [torch.cat((tkn[0],tkn[1])) for tkn in embedding], num_vectorize(target)


In [16]:
e,t = get_item(1)

In [17]:
t

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64)

In [18]:
len(t)

29

In [70]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [77]:
EMBEDDING_SIZE

371

In [79]:
len(e[0])

101

In [80]:
e[0][0].size()

torch.Size([371])

In [172]:
def pad_minibatch(minib):
    
    batchsize = len(minib)
    
    seq_lens = [len(emb) for emb in minib]
    max_len = max(seq_lens)
    
    # input of shape (seq_len, batch, input_size): tensor containing the features of the input sequence
    # https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
#     seq_tensor = torch.zeros((batchsize, max_len, EMBEDDING_SIZE)).double()
    seq_tensor = torch.zeros((max_len, batchsize, EMBEDDING_SIZE)).double()
    
                  
    for i, (seq,length) in enumerate( zip(minib,seq_lens) ):
        for wi,word in enumerate(seq):
#             seq_tensor[i,wi] = word
            seq_tensor[wi,i] = word

    return seq_tensor

In [173]:
pade = pad_minibatch(e)

In [174]:
pade.size()

torch.Size([132, 5, 371])

In [129]:
seq_lens_test[1]

41

In [132]:
pade[1].size()

torch.Size([132, 371])

In [135]:
len(e[1])

41

In [141]:
[e[1 for q in range(len(e[1]))]

TypeError: list indices must be integers or slices, not tuple

In [175]:
[torch.equal(pade[q,1],e[1][q]) for q in range(len(e[1]))]

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

In [133]:
pade[1,:41].size()

torch.Size([41, 371])

In [104]:
batchsize_test = len(e)

In [100]:
seq_lens_test = [len(emb) for emb in e]

In [103]:
seq_lens_test

[101, 41, 132, 41, 66]

In [101]:
max_len_test = max(seq_lens_test)

In [102]:
max_len_test

132

In [151]:
# seq_tensor_test = torch.zeros((batchsize_test, max_len_test ,EMBEDDING_SIZE)).double()
seq_tensor_test2 = torch.zeros((max_len_test ,batchsize_test, EMBEDDING_SIZE)).double()


In [ ]:
for i, (seq, length) in enumerate(zip(e,seq_lens_test2)):
    iterator, s, l = i, seq, length
    break

In [108]:
for i, (seq, length) in enumerate(zip(e,seq_lens_test)):
    iterator, s, l = i, seq, length
    break

In [113]:
len(s)

101

In [119]:
for wi,word in enumerate(s):
    seq_tensor_test[i,wi] = word

In [122]:
seq_tensor_test[0].size()

torch.Size([132, 371])

In [117]:
seq_tensor_test[i,:seq_lens_test,] = s

ValueError: only one element tensors can be converted to Python scalars

In [106]:
seq_tensor.size()

torch.Size([5, 132, 371])

In [96]:
pade = pad_minibatch(e)

TypeError: can't assign a list to a torch.DoubleTensor

In [76]:
len(e)

5

In [66]:
#pad with zeros -- maybe
# end with eos tensor, tag

def get_minibatch(batchsize=1, ix=None):
    
    
    def num_vectorize(targets):
        vectors = []
        for i in targets:
            v = torch.zeros([30])
            v[:i] = 1
            vectors.append(v)
        return vectors
    
    
    minibatch_ix = random.sample(range(num_samples),batchsize) if ix is None else ix 
    
    #index, filename
    cookie_files = [(i,X_cookie_filenames[i]) for i in minibatch_ix if i in X_cookie_filenames.keys()]
    sent_files = [(i,X_sent_filenames[i] )for i in minibatch_ix if i in X_sent_filenames.keys()]
    
    embeddings = []
    targets = []

    for corpus,data,targetdict in [(cookie_files,X_cookie_reader,y_cookie),(sent_files,X_sentence_reader,y_sentence)]: 
        for file_ix,file in corpus:
#             print(file_ix, file)
#             print('Words',[tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])
#             print('Words_len',len([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance]))
            embedding = [(vocab_emb[token],torch.zeros(len(pos_dict),dtype=torch.float64),pos_dict[pos]) for (token,pos) in zip([tokensraw for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance], [pos for utterance in data.tagged_sents(participant='PAR',by_files=True)[file] for (tokensraw,pos,tokenstem,dependency) in utterance])]
            target = targetdict[file_ix]
            
            for tkn in embedding:

                tkn[1][tkn[2]] = 1

#                 embeddings.append(torch.cat((tkn[0],tkn[1])))
            
            embeddings.append([torch.cat((tkn[0],tkn[1])) for tkn in embedding])


#             embeddings.append(embedding)
            targets.append(target)
    print(targets)       
#     return embeddings, torch.tensor(targets), minibatch_ix
#     return embeddings, torch.tensor(targets)
    return embeddings, num_vectorize(targets)

In [68]:
e,t = get_minibatch(batchsize=5)

[30, 20, 11, 30, 19]


In [69]:
max([len(emb) for emb in e])

132

In [217]:
e,t = get_minibatch(batchsize=num_samples)

[20, 30, 26, 29, 19, 23, 28, 26, 23, 19, 29, 30, 29, 30, 29, 20, 30, 20, 29, 30, 21, 5, 30, 30, 29, 19, 15, 13, 29, 10, 16, 15, 30, 7, 30, 30, 28, 30, 30, 20, 28, 29, 13, 30, 26, 30, 24, 25, 24, 30, 25, 30, 19, 19, 19, 28, 24, 28, 30, 17, 28, 18, 30, 30, 27, 25, 30, 19, 29, 30, 26, 17, 10, 30, 28, 17, 30, 15, 27, 30, 30, 22, 30, 29, 27, 21, 30, 19, 15, 17, 27, 16, 30, 28, 24, 26, 19, 30, 29, 26, 15, 30, 30, 30, 22, 29, 28, 22, 30, 26, 30, 30, 28, 28, 30, 17, 20, 27, 25, 30, 28, 30, 20, 23, 30, 30, 30, 29, 29, 30, 20, 17, 19, 17, 15, 30, 21, 28, 22, 28, 29, 10, 28, 22, 22, 13, 30, 15, 29, 19, 28, 20, 30, 20, 29, 30, 30, 29, 30, 29, 29, 29, 30, 19, 19, 27, 18, 28, 28, 12, 30, 30, 12, 29, 30, 3, 30, 23, 19, 30, 11, 12, 19, 30, 24, 24, 29, 19, 16, 19, 30, 28, 25, 20, 30, 25, 20, 29, 10, 30, 30, 19, 30, 28, 15, 30, 29, 8, 20, 24, 30, 30, 28, 29, 28, 23, 30, 23, 29, 30, 30, 28, 16, 30, 30, 20, 26, 13, 28, 27, 30, 30, 30, 30, 23, 28, 18, 29, 24, 22, 30, 11, 13, 21, 30, 26, 13, 28, 20, 23, 23,

In [54]:
# https://towardsdatascience.com/simple-trick-to-train-an-ordinal-regression-with-any-classifier-6911183d2a3c
# validate -- accuracy 
def out_to_score_proba(yhat_vector):
    # 29-dim sized vector
    proba_vector = torch.zeros([30])
    # no zero-score
    proba_vector[0] = 0. 
    # probability == 1 , 1-P(y>1)   
    proba_vector[1] = 1-yhat_vector[0]
    
    for i in range(2,len(yhat_vector)):
        proba_vector[i] = yhat_vector[i-1] - yhat_vector[i]
        
    proba_vector[-1] = yhat_vector[-1] 
    
    print(proba_vector)
    return torch.argmax(proba_vector)
    if vector[0] == 0: return 1

In [51]:
yhat

tensor([0.5018, 0.4925, 0.4860, 0.4743, 0.4726, 0.4820, 0.5407, 0.5166, 0.5175,
        0.4928, 0.4907, 0.4837, 0.4799, 0.5198, 0.4877, 0.5155, 0.4944, 0.4801,
        0.4650, 0.5054, 0.5059, 0.4941, 0.5003, 0.5235, 0.5104, 0.4391, 0.4852,
        0.5216, 0.5149, 0.4370], dtype=torch.float64, requires_grad=True)

In [56]:
.4925 - .4860

0.006500000000000006

In [55]:
out_to_score_proba(yhat)

tensor([ 0.0000e+00,  4.9823e-01,  6.4766e-03,  1.1666e-02,  1.7491e-03,
        -9.4169e-03, -5.8660e-02,  2.4029e-02, -8.6583e-04,  2.4695e-02,
         2.1389e-03,  6.9371e-03,  3.8517e-03, -3.9920e-02,  3.2102e-02,
        -2.7816e-02,  2.1090e-02,  1.4348e-02,  1.5034e-02, -4.0413e-02,
        -4.9319e-04,  1.1883e-02, -6.2801e-03, -2.3158e-02,  1.3134e-02,
         7.1215e-02, -4.6066e-02, -3.6440e-02,  6.7199e-03,  4.3695e-01],
       grad_fn=<CopySlices>)


tensor(1)

In [27]:
t.size()

torch.Size([29])

In [28]:
out_to_score_proba(t)

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0.])

In [232]:
# accuracy measurement
# return summation -- doesnt perfectly handle skipped values
# np.where(vector)[0][-1] https://stackoverflow.com/questions/38375401/neural-network-ordinal-classification-for-age
def out_to_score_summation(vector):
    # zeros array 
    
    return vector.sum() + 1
    
    if len(np.where(vector)[0]) == 0: return 0
#     print ([i+2 for i in np.where(vector)])
    print(np.where(vector))
    candidate = (np.where(vector)[0][-1]) 
    
    # handling intermediate zeros...maybe not optimal
    if np.where(vector)[0][-2] < (candidate - 1): candidate = (np.where(vector)[0][-2] + np.where(vector)[0][-1])/2
        
    return candidate + 1
    # alternative
    return vector.sum() + 1
#     len(np.where(t)[0])

In [218]:
# batch of all samples:  longest sample length (for padding, if minibatch)
max([len(emb) for emb in e])

546

In [199]:
# all words same len of emb size 371 
set([len(emb[i]) for emb in e for i in range(len(emb))])

{371}

https://pytorch.org/tutorials/beginner/transformer_tutorial.html

Seq2Seq -- with attention

2 layer rnn..

In [30]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

371

In [31]:
HIDDEN_SIZE = 128
# EMBEDDING_SIZE = 371
# with EOS pos
EMBEDDING_SIZE = len(vocab_emb['I']) + len(pos_dict)

In [268]:
# max([len(emb) for emb in e])

546

In [32]:
# change if aadd in EOS, pos to 547
MAX_LENGTH = 546

In [33]:
VOCAB_SIZE = len(vocab_emb)

In [152]:
len(train_ix)

634

In [153]:
634/2

317.0

In [192]:
BATCH_SIZE = 32

In [193]:
len(train_ix)/32

19.8125

In [194]:
.8125*32

26.0

In [195]:
len(valid_ix)/32

4.9375

In [196]:
.9375*32

30.0

In [101]:
len(dict)

71

In [197]:
random.shuffle(train_ix)

In [199]:
random.shuffle(train_ix)

In [102]:
# pos_dict['EOS'] = len(pos_dict)

In [104]:
# EOS tag
# vocab_emb['EOS'] = torch.rand([300],dtype=torch.float64)

In [261]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size=MAX_LENGTH, emb_size=EMBEDDING_SIZE, hidden_size=HIDDEN_SIZE):
        super(EncoderRNN, self).__init__()
        
        self.emb_size = emb_size
        self.hidden_size = hidden_size

        # what is the input size fo self.embedding...? and is self.embedding necessary
#         self.embedding = nn.Embedding(input_size, emb_size)
        
        
        
#         self.gru = nn.GRU(hidden_size, hidden_size)
#         self.gru = nn.GRU(input_size=self.emb_size, hidden_size=self.hidden_size,num_layers=2)
# batch_first – If True, then the input and output tensors are provided as (batch, seq, feature) https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
        self.gru = nn.GRU(input_size=self.emb_size,hidden_size=self.hidden_size,num_layers=2).double()
#         # each output node Oi of our neural network
# uses a standard sigmoid function 1
# 1+e−zi
# , without including
# the outputs from other nodes, as shown in Figure 1. Output
# node Oi
# is used to estimate the probability oi
# that a data
# point belongs to category i independently, without subjecting
# to normalization as traditional neural networks do. Thus,
# for a data point x of category k, the target vector is
# (1, , 1, .., 1, 0, 0, 0), in which the first k elements is 1 and
# others 0
# http://orca.st.usm.edu/~zwang/files/rank.pdf A Neural Network Approach to Ordinal Regression
    
        self.fc = nn.Linear(self.hidden_size*2,30).double()
        
        self.activation = nn.Sigmoid()
#         self.activations_list = [nn.Sigmoid() for i in range(30)]

    def forward(self, input, hidden):
        # confirm dims here...
#         embedded = self.embedding(input).view(1, 1, -1)
        
        embedded = input.double()
#         embedded = torch.cat(input).view(len(input), batch_size, -1).double()
        
#         output = embedded
        
#         output, hidden = self.gru(self.emb_size, self.hidden_size)
#         output, hidden = self.gru(output)
        gru_out, _ = self.gru(embedded,hidden)
        
        hid_out = torch.cat((_[-2,:,:], _[-1,:,:]), dim = 1)
#         print(gru_out.size(),_.size())
    
#         lstm_out, _ = self.lstm(embedding.view(len(sentence), -1))
        out = self.fc(hid_out)
        print(out.size())
        
#         out = torch.tensor([a(o) for (a,o) in zip(self.activations_list,out[0])],requires_grad=True).double()
        out - self.activation(out)
        
        # any further processing -- int? softmax...? 
#         return out
        return out, _

    def initHidden(self,batch_size=BATCH_SIZE):
        # h_0 of shape (num_layers * num_directions, batch, hidden_size): tensor containing the initial hidden state for each element in the batch. If the LSTM is bidirectional, num_directions should be 2
        return torch.zeros(2, batch_size, self.hidden_size, device=device).double()
        


In [257]:
test_out = torch.ones([158,30])

In [258]:
m = nn.Sigmoid()
test_act = m(test_out)

In [259]:
test_act.size()

torch.Size([158, 30])

In [260]:
test_act

tensor([[0.7311, 0.7311, 0.7311,  ..., 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311,  ..., 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311,  ..., 0.7311, 0.7311, 0.7311],
        ...,
        [0.7311, 0.7311, 0.7311,  ..., 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311,  ..., 0.7311, 0.7311, 0.7311],
        [0.7311, 0.7311, 0.7311,  ..., 0.7311, 0.7311, 0.7311]])

In [262]:
encoder= EncoderRNN()

In [243]:
test_pack = nn.utils.rnn.pack_padded_sequence(pad_all_valid_x,lengths=valid_x_lens,enforce_sorted=False)

In [246]:
type(test_pack)

torch.nn.utils.rnn.PackedSequence

In [263]:
yhatpack, _ = encoder(test_pack,_hid_valid)

torch.Size([158, 30])


In [268]:
yhatpack.size()

torch.Size([158, 30])

In [272]:
yhatpack[157]

tensor([ 0.1788, -0.0023,  0.2569, -0.0905,  0.1914, -0.1614,  0.0171,  0.0897,
        -0.0817,  0.0992,  0.1001, -0.0084, -0.2938,  0.0060,  0.1528,  0.1018,
        -0.2014, -0.1314, -0.0788, -0.0804,  0.0092,  0.0939, -0.0202,  0.1876,
        -0.0398,  0.1013,  0.0640,  0.0624, -0.0851,  0.2401],
       dtype=torch.float64, grad_fn=<SelectBackward>)

In [274]:
[out_to_score_proba(y) for y in yhatpack[:10]]

tensor([ 0.0000,  0.9193, -0.0470,  0.1773, -0.2006,  0.2671, -0.1626,  0.0558,
        -0.1039,  0.1262, -0.2834,  0.1527,  0.2409, -0.3476, -0.0555,  0.1173,
         0.2131, -0.1395,  0.0643, -0.0094, -0.1291,  0.0731, -0.0127, -0.0782,
         0.1400,  0.0790, -0.1872,  0.1624,  0.0552,  0.1054],
       grad_fn=<CopySlices>)
tensor([ 0.0000,  0.8362, -0.2444,  0.3373, -0.2214,  0.2400, -0.0807, -0.0806,
         0.0299, -0.0234, -0.0919,  0.1324,  0.2497, -0.2795, -0.0936,  0.0769,
         0.2023, -0.0735, -0.0706, -0.0391, -0.0392, -0.0140,  0.0167,  0.0106,
         0.0349, -0.0715,  0.0303,  0.0116,  0.1582,  0.2454],
       grad_fn=<CopySlices>)
tensor([ 0.0000,  0.8810, -0.2009,  0.3188, -0.2852,  0.3235, -0.1724, -0.0179,
         0.0514, -0.0584, -0.0437,  0.0561,  0.3434, -0.3024, -0.1987,  0.0586,
         0.1978, -0.0372,  0.0277,  0.0192, -0.1223, -0.0166,  0.1178, -0.1830,
         0.0971, -0.0561,  0.0651,  0.0191,  0.1011,  0.0967],
       grad_fn=<CopySlices>)
tens

[tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1),
 tensor(1)]

In [265]:
yhatpack

tensor([[ 0.0807,  0.0318,  0.0788,  ..., -0.0805, -0.1357,  0.1054],
        [ 0.1638, -0.0103,  0.2341,  ...,  0.0406, -0.1176,  0.2454],
        [ 0.1190,  0.0086,  0.2095,  ...,  0.0080, -0.0931,  0.0967],
        ...,
        [ 0.1824,  0.0310,  0.3300,  ...,  0.0736, -0.0700,  0.3013],
        [ 0.1498, -0.0104,  0.2432,  ...,  0.0596, -0.0579,  0.2655],
        [ 0.1788, -0.0023,  0.2569,  ...,  0.0624, -0.0851,  0.2401]],
       dtype=torch.float64, grad_fn=<AddmmBackward>)

In [247]:
test_pack[0]

tensor([[ 0.0218, -0.2005,  0.1678,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1922,  0.2855,  0.0126,  ...,  0.0000,  1.0000,  0.0000],
        [ 0.0882,  0.4016, -0.0447,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.3946, -0.0134, -0.2329,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1088,  0.0022,  0.2221,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.1088,  0.0022,  0.2221,  ...,  0.0000,  0.0000,  0.0000]],
       dtype=torch.float64)

In [241]:
len(all_valid_x)

158

In [239]:
valid_x_lens = [len(x) for x in all_valid_x]

In [203]:
all_valid_x,all_valid_y = get_minibatch(ix=valid_ix)

[30, 18, 29, 28, 26, 27, 23, 30, 19, 17, 20, 17, 25, 30, 23, 29, 27, 30, 19, 16, 30, 7, 28, 28, 28, 28, 25, 12, 30, 18, 30, 25, 15, 18, 14, 26, 27, 30, 20, 17, 30, 26, 19, 27, 17, 20, 20, 29, 27, 29, 30, 30, 30, 20, 22, 29, 19, 30, 30, 30, 30, 22, 17, 27, 27, 30, 30, 23, 30, 30, 19, 25, 30, 19, 30, 30, 30, 8, 30, 22, 30, 30, 28, 16, 19, 22, 24, 28, 16, 30, 30, 19, 17, 27, 19, 30, 30, 21, 28, 19, 29, 19, 12, 28, 20, 23, 13, 20, 23, 27, 23, 19, 20, 19, 19, 19, 27, 16, 10, 20, 17, 24, 24, 23, 16, 20, 18, 17, 19, 28, 29, 20, 27, 20, 23, 16, 20, 23, 27, 22, 19, 20, 20, 19, 19, 28, 28, 28, 26, 19, 24, 22, 20, 18, 23, 27, 25, 25]


In [219]:
encoder = EncoderRNN()

In [ ]:
_hid_

In [210]:
pad_all_valid_x = pad_minibatch(all_valid_x)

In [212]:
pad_all_valid_x.size()

torch.Size([312, 158, 371])

In [213]:
len(valid_ix)

158

In [220]:
_hid_valid = encoder.initHidden(batch_size=len(valid_ix))

In [227]:
yhat_valid, valid_hid = encoder(pad_all_valid_x,_hid_valid)

In [231]:
_hid_valid.size()

torch.Size([2, 158, 128])

In [232]:
valid_hid.size()

torch.Size([2, 158, 128])

In [233]:
yhat_valid.size()

torch.Size([30])

In [267]:
yhatpack

tensor([[ 0.0807,  0.0318,  0.0788,  ..., -0.0805, -0.1357,  0.1054],
        [ 0.1638, -0.0103,  0.2341,  ...,  0.0406, -0.1176,  0.2454],
        [ 0.1190,  0.0086,  0.2095,  ...,  0.0080, -0.0931,  0.0967],
        ...,
        [ 0.1824,  0.0310,  0.3300,  ...,  0.0736, -0.0700,  0.3013],
        [ 0.1498, -0.0104,  0.2432,  ...,  0.0596, -0.0579,  0.2655],
        [ 0.1788, -0.0023,  0.2569,  ...,  0.0624, -0.0851,  0.2401]],
       dtype=torch.float64, grad_fn=<AddmmBackward>)

In [266]:
[out_to_score_proba(y) for y in yhat_valid]

IndexError: invalid index of a 0-dim tensor. Use tensor.item() to convert a 0-dim tensor to a Python number

In [226]:
len(yhat_valid[0])

30

In [229]:
yhat_valid

tensor([0.5087, 0.4875, 0.5120, 0.4910, 0.4997, 0.4945, 0.5163, 0.5021, 0.4805,
        0.5039, 0.4823, 0.5038, 0.5086, 0.5047, 0.5043, 0.4801, 0.5084, 0.5078,
        0.5105, 0.4916, 0.4864, 0.4858, 0.5025, 0.4963, 0.4971, 0.4851, 0.4880,
        0.4966, 0.4863, 0.5173], dtype=torch.float64, requires_grad=True)

In [155]:
x,y = get_item()

In [158]:
len(x)

55

In [160]:
torch.cat(x).view(len(x),1,-1).double().size()

torch.Size([55, 1, 371])

In [35]:
# The
# cost function for a data point x can be relative entropy
# or square error between the target vector and the output
# vector
# loss = nn.BCELoss()
loss_func = nn.MSELoss()

# TODO minibatch + pad. For starters, stochastic gd..

In [188]:
encoder = EncoderRNN()

In [165]:
optimizer = torch.optim.Adam(encoder.parameters(), lr=1e-2)

In [189]:
_hidden = encoder.initHidden()

In [176]:
embs = pad_minibatch(e)

In [177]:
embs.size()

torch.Size([132, 5, 371])

In [168]:
embs.size()

torch.Size([5, 132, 371])

In [190]:
yhat, _hid = encoder(embs,_hidden)

In [191]:
yhat

tensor([0.4916, 0.4928, 0.5082, 0.4901, 0.4774, 0.4977, 0.4997, 0.4825, 0.4995,
        0.4957, 0.4839, 0.5076, 0.4890, 0.4779, 0.4804, 0.4845, 0.5070, 0.5043,
        0.4711, 0.5178, 0.5031, 0.4908, 0.4881, 0.4853, 0.4942, 0.4968, 0.4961,
        0.4831, 0.4917, 0.4878], dtype=torch.float64, requires_grad=True)

In [41]:
y

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=torch.float64)

In [40]:
x,y = get_item()

In [41]:
yhat, hid = encoder(x,_hidden)

In [42]:
yhat

tensor([0.5018, 0.4925, 0.4860, 0.4743, 0.4726, 0.4820, 0.5407, 0.5166, 0.5175,
        0.4928, 0.4907, 0.4837, 0.4799, 0.5198, 0.4877, 0.5155, 0.4944, 0.4801,
        0.4650, 0.5054, 0.5059, 0.4941, 0.5003, 0.5235, 0.5104, 0.4391, 0.4852,
        0.5216, 0.5149, 0.4370], dtype=torch.float64, requires_grad=True)

In [43]:
out_to_score_proba(yhat)

tensor(1)

In [49]:
loss_func(y,yhat)

tensor(0.2627, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [50]:
encoder.train()
_hidden = encoder.initHidden()


for epoch in range(10):
    print('Epoch',epoch)
    step = 0
    
    # in future, mix up epochs 
    for i in train_ix:
        
        step += 1
        optimizer.zero_grad()
        
        x,y = get_item(i)

        yhat, _hidden = encoder(x,_hidden)

        loss = loss_func(yhat,y)

        loss.backward()
#         loss.backward(retain_graph=True)
        
        optimizer.step()

        if step % 20 == 0: 
            print('step',step)
            print('loss',loss)

    
    
    

Epoch 0
step 20
loss tensor(0.2631, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 40
loss tensor(0.2647, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 60
loss tensor(0.2632, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 80
loss tensor(0.2630, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 100
loss tensor(0.2621, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 120
loss tensor(0.2596, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 140
loss tensor(0.2642, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 160
loss tensor(0.2613, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 180
loss tensor(0.2605, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 200
loss tensor(0.2449, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 220
loss tensor(0.2631, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 240
loss tensor(0.2515, dtype=torch.float64, grad_fn=<MseLossBackward>)
step 260
loss tensor(0.2556, dtype=torch.float64, grad_fn=<MseLossBackwa

In [290]:
x,y = get_minibatch(batchsize=1)

In [287]:
torch.cat(x[0]).view(len(x[0]),1, -1).shape

torch.Size([55, 1, 371])

In [251]:
# need to be able to pass any/all into net
# encoder = EncoderRNN(len(x[0])).to(device)

In [293]:
_hidden.size()

torch.Size([2, 1, 128])